In [298]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, String, Integer, Float, Date, Table, ForeignKey
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, relationship

import pandas as pd

In [299]:
credentials = pd.read_csv('../credentials/credentials.csv')

database_adress = credentials['Host'].iloc[0]
database_user = credentials['User'].iloc[0]
database_password = credentials['Password'].iloc[0]
database_name = credentials['Database'].iloc[0]
database_port = credentials['Port'].iloc[0]

print(database_user)

darth_vader


In [300]:
Base = automap_base()

# Create the middle Table movie_genres which is between the tables movies and genres which is am many to many relationship
movie_genres = Table(
    "movies_genres",
    Base.metadata,
    Column("movieId", ForeignKey("movies.movieId")),
    Column("genreId", ForeignKey("genres.genreId")),
)

#Creation of the class Genre(Tabele name 'genres') with it's 3 properties genre_id, name and all the movies that are assigned to that genre
#Movies is a many to many relation because every genre can have multiple movies with it's genre and every movie can have multiple genres
class Genre(Base):
   __tablename__ = 'genres' 
   genre_id = Column('genreId', Integer, primary_key = True)
   name = Column('genrename', String)
   movies = relationship('Movie', secondary=movie_genres,
                            backref='movies', viewonly=True)

#Creation of the class Genome_Tag(Table name 'genometags') with it's two properties tag_id and tag
class Genome_Tag(Base):
    __tablename__ = 'genometags' 
    tag_id = Column('tagId', Integer, primary_key = True)
    tag = Column('tag', String)

#Creation of the class Link(Table name 'links') with it's three properties movie, imdb_id and tmdb_id.
#Movie is the primary key of the table and at the same time the foreign key to the table movie
class Link(Base):
    __tablename__ = 'links' 
    movieId = Column(Integer, ForeignKey("movies.movieId"), primary_key = True)
    movie = relationship("Movie", back_populates="link", viewonly=True)
    imdb_id = Column('imdbId', Integer)
    tmdb_id = Column('tmdbId', Integer)

#Creation of the class Rating(Table name 'ratings') with it's five properties rating_id, user_id, movie, rating, rating_date
#Movie is a one to many relationship to Rating, which means that every rating rates one movie and every movie can have multiple ratings
class Rating(Base):
    __tablename__ = 'ratings' 
    rating_id = Column('rating_id', Integer, primary_key = True)
    user_id = Column('userId', Integer)
    movieId = Column(Integer, ForeignKey("movies.movieId"))
    movie = relationship("Movie", back_populates="rating", viewonly=True)
    rating = Column('rating', Float)
    rating_date = Column('rating_date', Date)

#Creation of the class Tag(Table name 'tags') with it's five properties tag_id, user_id, movie, tag, tag_date
#Movie is a one to many relationship to Tag, which means that every tag belongs to one movie and every movie can have multiple tags
class Tag(Base):
    __tablename__ = 'tags' 
    tag_id = Column('tag_id', Integer, primary_key = True)
    user_id = Column('userId', Integer)
    movieId = Column(Integer, ForeignKey("movies.movieId"))
    movie = relationship("Movie", back_populates="tag", viewonly=True)
    tag = Column('tag', String)
    tag_date = Column('tag_date', Date) 

#Creation of the class Genome_Score(Table name 'genomescores') with it's three properties tag_id, movie, relevance
#Genome_Score has two primary keys because the combination of tag_id and movieId is unique to every entry
#Movie is a many to many relationship to Genome_Score, which means that every genome_score belongs to every movie and every movie has a relevance rating of every tag
class Genome_Score(Base):
    __tablename__ = 'genomescores' 
    tag_id = Column('tagId', Integer, primary_key = True)
    movieId = Column(Integer, ForeignKey("movies.movieId"), primary_key = True)
    movie = relationship("Movie", back_populates="genome_score", viewonly=True)
    relevance = Column('relevance', Float)
    

#Creation of the class Movie(Table name 'movies') with it's eight properties movie_id, title, release_year, genres, ratings, tags, genome_scores, link
#Genres is a many to many relation because every movie can have multiple genres with it's and every genre can have multiple movies
#Ratings is many to one relation to Movie, because every movie can have multiple ratings and every rating belongs to one movie
#Tags is many to one relation to Movie, because every movie can have multiple tags and every tag belongs to one movie
#Genome_Scores is many to many relation to Movie with two primary keys, because every movie has a relevance rating to every tag so the combination of movieId an tagId is unique for every entry
class Movie(Base):
    __tablename__ = 'movies'

    movie_id = Column('movieid', Integer, primary_key = True)
    title = Column('title', String)
    release_year = Column('release_year', Integer)

    genres = relationship('Genre', secondary=movie_genres,
                            backref='genres', viewonly=True)
    ratings = relationship("Rating" ,back_populates='movie', viewonly=True)
    tags = relationship("Tag" ,back_populates='movie', viewonly=True)
    genome_scores = relationship("Genome_Score" ,back_populates='movie', viewonly=True)
    link = relationship("Link" ,back_populates='movie', viewonly=True, uselist=False)

In [301]:
#Build the connection with the database
engine = create_engine(f'mysql+pymysql://{database_user}:{database_password}@{database_adress}:{database_port}/{database_name}')
Base.prepare(autoload_with=engine)

session = Session(engine)

In [302]:
# Function which prints out all the information about a movie
def print_Movie_Infos(movie):

    ratings = list(movie.ratings)
    tags = list(movie.tags)
    genome_scores = list(movie.genome_scores)

    print(f'Movie id: {movie.movie_id}')
    print(f'Title: {movie.title}')
    print(f'Release Year: {movie.release_year}')

    print()
    print("Genres: ")
    for genre in list(movie.genres):
        print(genre.name)

    print()
    print(f'IMDb id: {movie.link.imdb_id}')
    print(f'TMDb id: {movie.link.tmdb_id}')

    print()
    print(f'First 10 ratings of movie {movie.title}')

    i_ratings = 0

    while i_ratings < 10:
        print(ratings[i_ratings].rating)
        i_ratings += 1

    print()
    print(f'First 10 tags of movie {movie.title}')

    i_tags = 0

    while i_tags < 10:
        print(tags[i_tags].tag)
        i_tags += 1

    print()
    print(f'First 10 genome_scores of movie {movie.title}')

    i_genome_scores = 1

    while i_genome_scores < 10:
        print(genome_scores[i_genome_scores].relevance)
        i_genome_scores += 1

In [303]:
# Function which prints out all the information about a genome_Tag
def print_Genome_Tag_Infos(genome_tag):
    print(f'Genome tag id: {genome_tag.tag_id}')
    print(f'Genome tag: {genome_tag.tag}')

In [304]:
# Function which prints out all the information about a link
def print_Link_Infos(link):
    print(f'Movie title: {link.movie.title}')
    print(f'IMDb id: {link.imdb_id}')
    print(f'TMDb id: {link.tmdb_id}')

In [305]:
# Function which prints out all the information about a rating
def print_Rating_Infos(rating):
    print(f'Rating id: {rating.rating_id}')
    print(f'Rating user id: {rating.user_id}')
    print(f'Rating movie title: {rating.movie.title}')
    print(f'Rating: {rating.rating}')
    print(f'Rating date: {rating.rating_date}')

In [306]:
# Function which prints out all the information about a tag
def print_Tag_Infos(tag):
    print(f'Tag id: {tag.tag_id}')
    print(f'Tag user id: {tag.user_id}')
    print(f'Tag movie title: {tag.movie.title}')
    print(f'Tag: {tag.tag}')
    print(f'Tag date: {tag.tag_date}')


In [307]:
# Function which prints out all the information about a genome_Score
def print_Genome_Score_Infos(genome_score):
    print(f'Genome score tag id: {genome_score.tag_id}')
    print(f'Genome score movie title: {genome_score.movie.title}')
    print(f'Genome score relevance: {genome_score.relevance}')

In [308]:
# Function which prints out all the information about a genre
def print_Genre_Infos(genre):
    print(f'Genre id: {genre.genre_id}')
    print(f'Genre name: {genre.name}')
    print(f'First 10 movies with the genre {genre.name}: {genre.genre_id}')

    movies = list(genre.movies)
    i_movies = 0

    while i_movies < 10:
        print(movies[i_movies].title)
        i_movies += 1    


In [309]:
movie_example = session.query(Movie).first()
print_Movie_Infos(movie_example)

Movie id: 1
Title: Toy Story (1995)
Release Year: 1995

Genres: 
Adventure
Animation
Children
Comedy
Fantasy

IMDb id: 0114709
TMDb id: 862

First 10 ratings of movie Toy Story (1995)
4.0
5.0
4.5
4.0
4.0
4.0
5.0
5.0
4.5
5.0

First 10 tags of movie Toy Story (1995)
animated
buddy movie
Cartoon
cgi
comedy
computer animation
family
friendship
kids
toy

First 10 genome_scores of movie Toy Story (1995)
0.02375
0.05425
0.06875
0.16
0.19525
0.076
0.252
0.2275
0.024


In [310]:
genome_tag_example = session.query(Genome_Tag).first()
print_Genome_Tag_Infos(genome_tag_example)

Genome tag id: 1
Genome tag: 007


In [311]:
link_example = session.query(Link).first()
print_Link_Infos(link_example)

Movie title: Toy Story (1995)
IMDb id: 0114709
TMDb id: 862


In [312]:
rating_example = session.query(Rating).first()
print_Rating_Infos(rating_example)

Rating id: 1
Rating user id: 1
Rating movie title: Three Colors: Blue (Trois couleurs: Bleu) (1993)
Rating: 3.5
Rating date: 2009-10-27 09:00:21


In [313]:
tag_example = session.query(Tag).first()
print_Tag_Infos(tag_example)

Tag id: 1
Tag user id: 14
Tag movie title: Braveheart (1995)
Tag: epic
Tag date: 2015-09-24 14:35:38


In [314]:
genome_score_example = session.query(Genome_Score).first()
print_Genome_Score_Infos(genome_score_example)

Genome score tag id: 1
Genome score movie title: Stranger by the Lake (L'inconnu du lac) (2013)
Genome score relevance: 0.005


In [315]:
genre_example = session.query(Genre).first()
print_Genre_Infos(genre_example)

Genre id: 1
Genre name: Action
First 10 movies with the genre Action: 1
Heat (1995)
Sudden Death (1995)
GoldenEye (1995)
Cutthroat Island (1995)
Money Train (1995)
Assassins (1995)
Dead Presidents (1995)
Mortal Kombat (1995)
Guardian Angel (1994)
Lawnmower Man 2: Beyond Cyberspace (1996)
